We will be researching the Cart-pole problem with a policy gradient agorithim

We Must first build our enviorment that are reinforcement learning model will learn in

In [34]:
import mujoco
import mujoco_viewer
import matplotlib.pyplot as plt

Now let's create the neccesary functions for our enviorment

Our model will take in 4 values: X value, X velocity, Theta value, Theta velocity.

We will need the enviorment randomized every new episode played by the model

In [35]:
import random as rand
import math
def Intial_state():
    theta = rand.uniform(-math.radians(6), math.radians(6))
    xpos = rand.uniform(-0.05, 0.05)
    xdot = rand.uniform(-0.5, 0.5)
    theta_dot = 0
    return xpos, xdot, theta, theta_dot

Now lets make the neccessary functions for our model. It needs to pushed to the left and to the right. We will use the data.crtl[0] to use the simulated acctuators that are in our model. The data element is all of the data of the cartpole at any moment this includes X Velo, theta Velo, X Pos, Theta Pos

In [36]:
import numpy as np
FORCE_MAG = 2.0

def right(data, force=FORCE_MAG):
    data.ctrl[0] = np.clip(force, -2.0, 2.0)


def left(data, force=FORCE_MAG):
    data.ctrl[0] = np.clip(-force, -2.0, 2.0)

Our model will need to know the state it is in, so we will create a function to get the raw values of the state for the model

In [37]:
def gather_raw_state(data):
    xpos = data.qpos[0]
    xdot = data.qvel[0]
    theta = data.qpos[1]
    theta_dot = data.qvel[1]
    return np.array([xpos, xdot, theta, theta_dot], dtype=np.float32)

We also need a function to normalize the state to make it easeier for the model to make outputs

In [38]:
def normalize_state(raw_state):
    x, x_dot, theta, theta_dot = raw_state
    sin_theta = math.sin(theta)
    cos_theta = math.cos(theta)
    norm_x = np.clip(x / 1.0, -1.0, 1.0)
    norm_x_dot = np.tanh(x_dot)
    norm_theta_dot = np.tanh(theta_dot)
    return np.array([norm_x, norm_x_dot, sin_theta, cos_theta, norm_theta_dot], dtype=np.float32)

Now we will create the reward funcition for the model

In [39]:
#Limits to determine if the episode is done
THETA_LIMIT_RADIANS = math.radians(12)
X_LIMIT = 1.0

In [40]:
def compute_reward(raw_state):
    x, x_dot, theta, theta_dot = raw_state
    angle_error = abs(theta)

    position_error = abs(x)
    reward = 1.0
    reward -= 2.0 * (angle_error / THETA_LIMIT_RADIANS)
    reward -= 0.5 * (position_error / X_LIMIT)
    reward -= 0.01 * (abs(x_dot) + abs(theta_dot))
    return max(reward, -2.0)


Now we will make a function to check if an episode is done

In [41]:
def episode_finished(raw_state):
    x, _, theta, _ = raw_state
    return abs(theta) > THETA_LIMIT_RADIANS or abs(x) > X_LIMIT

Now we must create our Nueral Network for the cartpole problem
We will be creating a nueral netowrk through pytorch. It will take in 4 dimensions as an input for the network: X Value, X Velocity, Theta value, Theta Velocity

We will have 3 layers in the netowrk, the first will take in the state layer, second being an input from the frist, and the thrid being an input from the second and outputting an action from our action set. Our action set is either sliding to the left or sliding to the right.

In [42]:
import torch
import torch.nn as nn

class PolicyNetwork(nn.Module):
    def __init__(self,state_dim = 5, action_dim = 2, hidden_dim = 128):
        super().__init__()
        self.layer1 = nn.Linear(state_dim, hidden_dim)
        self.layer2 = nn.Linear(hidden_dim, hidden_dim)
        self.output_layer = nn.Linear(hidden_dim, action_dim)

    def forward(self, x):
        x = torch.relu(self.layer1(x))
        x = torch.relu(self.layer2(x))
        return self.output_layer(x)  # logits (unnormalized)


We will also make a sample action function for the nueral network

In [43]:
def sample_action(policy_net, state):
    state_t = torch.as_tensor(state, dtype=torch.float32).unsqueeze(0)
    logits = policy_net(state_t)  # shape [1, action_dim]
    dist = torch.distributions.Categorical(logits=logits)
    action = dist.sample()  # shape [1]
    log_prob = dist.log_prob(action)  # shape [1]
    entropy = dist.entropy()  # shape [1] - PROPER entropy over all actions
    return action.item(), log_prob.squeeze(0), entropy.squeeze(0)


In [44]:
def compute_discounted_returns(rewards, gamma=0.99):
    """
    Compute discounted returns WITHOUT normalization.
    Normalization within an episode destroys the learning signal!
    """
    discounted_returns = []
    R = 0
    for r in reversed(rewards):
        R = r + gamma * R
        discounted_returns.insert(0, R)
    return torch.FloatTensor(discounted_returns)

Code to visualize total reward over time

In [45]:
def plot_rewards(total_rewards_over_time):
    plt.figure(figsize=(8, 5))
    plt.plot(total_rewards_over_time, label='total_reward')
    plt.xlabel('Episode')
    plt.ylabel('Total Reward')
    plt.title('Total Reward Over Time')
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.savefig('total_rewards_over_time_plot.png', dpi=200)
    plt.close()

Now lets add our necessary constants for training

In [46]:
RENDER_TRAINING = False
MODEL_PATH = 'cartpole.xml'
EPISODES = 300
MAX_STEPS = 1000

# Training hyperparameters
ENTROPY_BETA = 0.005  # encourages exploration


Lets setup the enviorment for the cartpole problem

In [47]:
model = mujoco.MjModel.from_xml_path(MODEL_PATH)
data = mujoco.MjData(model)
if RENDER_TRAINING:
    viewer = mujoco_viewer.MujocoViewer(model, data)
else:
    viewer = None

Setting up model and optimizers for this

In [48]:
policy_net = PolicyNetwork()
optimizer = torch.optim.Adam(policy_net.parameters(), lr=1e-3)

# Running baseline for variance reduction (proper way to normalize)
baseline_returns = []

Now lets Make the training loop for policy gradient

In [ ]:
total_rewards_overtime = []
for i in range(EPISODES):
    #initialize the environment for each episode to have a random state
    data.qpos[0], data.qvel[0], data.qpos[1], data.qvel[1] = Intial_state()
    data.ctrl[:] = 0
    #Total reward is used to graph
    total_reward = 0.0

    rewards = []

    #Lists to store states, actions, and log probabilities for each step
    log_probs = []
    entropies = []

    #Getting states
    raw_state = gather_raw_state(data)
    state = normalize_state(raw_state)

    #Now lets iterate through the steps of the episode
    for step in range(MAX_STEPS):

        if RENDER_TRAINING:
            viewer.render()
        
        #Getting the action from the policy network
        action, log_prob, entropy = sample_action(policy_net, state)
        #Storing the log probability and entropy of the action for training later
        log_probs.append(log_prob)
        entropies.append(entropy)
        
        if action == 0:
            left(data)
        else:
            right(data)

        #Step the simulation forward
        mujoco.mj_step(model, data)
        next_raw_state = gather_raw_state(data)
        next_state = normalize_state(next_raw_state)

        #Compute the reward and check if the episode is done
        reward = compute_reward(next_raw_state)
        
        #Accumulate the rewards BEFORE checking done
        total_reward += reward
        rewards.append(reward)
        
        if episode_finished(next_raw_state):
            break

        #setting the state to the next state
        state = next_state
        raw_state = next_raw_state
    
    # Compute discounted returns (NO normalization within episode!)
    returns = compute_discounted_returns(rewards, gamma=0.99)
    
    # Optional: subtract running baseline for variance reduction
    # This is the CORRECT way to use a baseline - compare across episodes, not within
    if len(baseline_returns) > 0:
        baseline = sum(baseline_returns) / len(baseline_returns)
        returns = returns - baseline
    
    # Update baseline with this episode's returns
    baseline_returns.append(returns[0].item())  # Store the initial return
    if len(baseline_returns) > 100:  # Keep last 100 episodes
        baseline_returns.pop(0)
    
    # Stack log_probs and entropies into tensors
    log_probs_t = torch.stack(log_probs)  # shape [T]
    entropies_t = torch.stack(entropies)  # shape [T]
    
    # Compute policy loss with proper entropy bonus
    policy_loss = -(log_probs_t * returns).sum()
    entropy_bonus = entropies_t.sum()  # sum of entropies over trajectory
    loss = policy_loss - ENTROPY_BETA * entropy_bonus

    optimizer.zero_grad()
    loss.backward()
    torch.nn.utils.clip_grad_norm_(policy_net.parameters(), max_norm=1.0)
    optimizer.step()

    total_rewards_overtime.append(total_reward)
    
    # Print progress every 10 episodes
    if (i + 1) % 10 == 0:
        avg_reward = sum(total_rewards_overtime[-10:]) / 10
        print(f"Episode {i+1}/{EPISODES}, Avg Reward (last 10): {avg_reward:.2f}, Total Reward: {total_reward:.2f}")
    
    plot_rewards(total_rewards_overtime)

Episode 10/300, Avg Reward (last 10): 3.39, Total Reward: -1.67
Episode 20/300, Avg Reward (last 10): 2.84, Total Reward: 1.37
Episode 20/300, Avg Reward (last 10): 2.84, Total Reward: 1.37
Episode 30/300, Avg Reward (last 10): 4.95, Total Reward: 17.61
Episode 30/300, Avg Reward (last 10): 4.95, Total Reward: 17.61
Episode 40/300, Avg Reward (last 10): 8.33, Total Reward: 21.84
Episode 40/300, Avg Reward (last 10): 8.33, Total Reward: 21.84
Episode 50/300, Avg Reward (last 10): -1.32, Total Reward: 7.62
Episode 50/300, Avg Reward (last 10): -1.32, Total Reward: 7.62
Episode 60/300, Avg Reward (last 10): 3.91, Total Reward: -5.43
Episode 60/300, Avg Reward (last 10): 3.91, Total Reward: -5.43
Episode 70/300, Avg Reward (last 10): 3.01, Total Reward: 9.35
Episode 70/300, Avg Reward (last 10): 3.01, Total Reward: 9.35
Episode 80/300, Avg Reward (last 10): 3.62, Total Reward: -2.74
Episode 80/300, Avg Reward (last 10): 3.62, Total Reward: -2.74
Episode 90/300, Avg Reward (last 10): 7.83, 